# Introduction  

For this particular setup, the atmosphere is parameterized within the "chemically-consistent" framework as described in Kreidberg et al. 2015.  The atmospheric composition is parameterized with only a metalicity and carbon-to-oxygen ratio assumed to be in thermochemical equilibrium along the temperature-pressure profile.  Originally this code would compute the gas and condensate phase mixing ratios by calling the NASA CEA routine. However, in order to remove fortran dependencies, a finely sampled pre-computed, interpolateable chemistry grid was instead produced with CEA as a function of temperature ($log_{10}(T)$ from 2.6 (398K) - 3.475 (2985K) in 0.025 increments), pressure ($log_{10}(P)$ from -7.0 (0.1$\mu$bar) - 2.4 (316 bar) in 0.1 increments), metallicity ($[M/H]$ from -1.5 (0.03$\times$) to 3.3 (2000$\times$)), and C/O ($log_{10}(C/O)$ from -1.0 (0.1) to 0.3 (2) awkwardly spaced to better sample the transition about C/O=1). All elemental abundances are scaled with respect to the Lodders 2009 solar abundance pattern.  A pseudo-hack rainout approximation is made to the gas phase abundances of TiO, VO, Na, K, and FeH.  In this hack, these species are set to 0 abundance at levels above where they first fall below some critical value ($10^{-11}$). This is to mimic the loss of these species from the gas phase into the condensate phase. In no case are we accounting for the loss of elemental abundances.

The 3-parameter temperature profile parameterization utilizes the Guillot 2010/Parmentier et al. 2014 analytic formulism (see Line et al. 2013a for implementation details).     

The transmission spectrum routine closely follows the equations (and figure) in Tinetti et al. 2012.  Instead of using line-by-line, or "sampled" cross-sections, this implementation uses the "correlated-K" method (see Lacis & Oinas 1990, or more recently Amundsen et al. 2016). Correlated-K is advantageous as it preserves the wavelength bin"integrated"  precision as line-by-line but with far less demanding computation.  We include as correlated-K line opacites H2O, CH4, CO, CO2, NH3, HCN, H2S, C2H2, Na, K, TiO, and VO and as continuum gas opacities H2-H2 and H2-He CIA. 

To handle the effects of disequilibrium chemistry due to vertical mixing, we apply the "quench-pressure" approximation.  We include a quench pressure parameter for the carbon-system and one for the nitrogen system.  The carbon quench pressure fixes the H2O, CO, and CH4 abundances above the quench pressure level to their abundances at the quench pressure level.  Similarly, the nitrogen quench pressure fixes the N2, NH3, and HCN abundances above the quench pressure to their values at the quench pressure level.  This is indeed a kludge, and a better implementation would be to use the timescale/eddy mixing prescription described in Zahnle & Marley 2015.  Regardless, any non-full kinetics approach is a kludge anyway.  

There are two different cloud prescriptions built in.  The first is the Ackerman & Marley 2001 "eddy-sed" approach that self-consistently computes the vertical particle size distribution given a sedimentation factor, $f_{sed}$ and an eddy mixing factor (K$_zz$) from some cloud base pressure and intrinsic condensate mixing ratio.  The classic "power-law haze" and "grey cloud" prescripton is also included.

Finally, if one doesn't like the "chemically-consistent" concept, they can use the "gas_scale" array to switch off or scale the abundances each opacity source.  

This specific notebook goes through the steps to generate a synthetic HST WFC3 dataset (for which we know the "true" answer) and then retrieves on it.


Software Requirements: This runs in the python 3 anaconda environment.  It is also crucial that anaconda numba is installed as many of the routines are optimized using numba's "@jit" decorator (http://numba.pydata.org/). 


# Import Routines, Load Opacities  

This first segment loads in the routines from fm.py and the correlated-K coefficients.  There are two sets of correlated-K coefficients (which I've called "xsecs" here).  There are ones taylored for HST WFC3+STIS (xsects_HST function in fm.py) and JWST (xsects_JWST in fm.py).  The WFC3+STIS correlated-K coefficients are generated at an R=200 longwards of 1 $\mu$m (up to 5$\mu$m and R=500 from 0.3 - 1 $\mu$m. The JWST CK-coefficients are generated at a constant R=100 from 0.8 - 11 $\mu$m.  Note...these *are not sampled cross-sections* so each resolution element at that R is correctly computed and matches line-by-line when binned to that same R.

Note that the "core" set of routines are all in fm.py.  If you want to know more about what is in the sausage, look into fm.py.  

In [ ]:
#import all of the functions in fm, namely, the CK-coefficients (may take a minute)
from fm import *
#preload CK-coeffs--a giant array/variable to be passed--inputs are lower wavenumber, upper wavenumber
#between 2000 and 30000 cm-1 for HST--R=200 > 1 um, then R=500 < 1 um 
#between 900 and 12000 cm-1 for JWST--R=100 across full wl range
#to convert between microns and wavenumbers-- wavelength [um] = 10,000/wavenumber [cm-1]
#make sure xsec wavenumber/wavelength range is *larger* than data wavelength range
xsecs=xsects_HST(2000, 30000)
#xsecs=xsects_JWST(1000, 10000)


# Setup Atmospheric Parameters to Generate a Spectrum

This segement defines the various atmospheric quantities and assignes them values for the generation of a simple transmission spectrum.  A description of each parameter along with a reasonable range of values is given as a comment following the assigned value. All of the parameters are then put into the parameter "state-vector" array, x.

In [ ]:
#setup "input" parameters. We are defining our 1D atmosphere with these
#the parameters
#planet/star system params--xRp is the "Rp" free parameter, M right now is fixed, but could be free param
Rp= 1.10   # Planet radius in Jupiter Radii--this will be forced to be 1 bar radius--arbitrary (scaling to this is free par)
Rstar=1.20  #Stellar Radius in Solar Radii
M =1.0   #Mass in Jupiter Masses
xRp=1.0  #scaling factor to Radius

#TP profile params (3--Guillot 2010, Parmentier & Guillot 2013--see Line et al. 2013a for implementation)
Tirr=1400     #Irradiation temperature as defined in Guillot 2010
logKir=-0.5  #TP profile IR opacity controlls the "vertical" location of the gradient
logg1=-1     #single channel Vis/IR opacity. Controls the delta T between deep T and TOA T

#Composition parameters---assumes "chemically consistent model" described in Kreidberg et al. 2015
logMet=0.0  #.   #Metallicity relative to solar log--solar is 0, 10x=1, 0.1x = -1: valid range is -1.5 - 3.0
logCtoO=-0.26   #log C-to-O ratio: log solar is -0.26: valid range is -1.0 - 0.3 
logPQCarbon=-5.5  #CH4, CO, H2O Qunech pressure--forces CH4, CO, and H2O to constant value at quench pressure value: valid range -6.0 - 1.5
logPQNitrogen=-5.5  #N2, NH3 Quench pressure--forces N2 and NH3 to ""  

#Ackerman & Marley 2001 Cloud parameters--physically motivated with Mie particles
logKzz=9 #log Kzz (cm2/s)--valid range: 2 - 11 -- higher values make larger particles
fsed=1.0 #sediminetation efficiency--valid range: 0.5 - 5--lower values make "puffier" more extended cloud 
logPbase=1.5  #cloud base pressure--valid range: -6.0 - 1.5
logCldVMR=-15.0 #cloud condensate base mixing ratio (e.g, see Fortney 2005)--valid range: -15 - -2.0

#simple 'grey+rayleigh' parameters just in case you don't want to use a physically motivated cloud
#(most are just made up anyway since we don't really understand all of the micro-physics.....)
logKcld = -40  #uniform in altitude and in wavelength "grey" opacity (it's a cross-section)--valid range: -50 - -10 
logRayAmp = -30  #power-law haze amplitude (log) as defined in des Etangs 2008 "0" would be like H2/He scat--valid range: -30 - 3 
RaySlope = 0  #power law index 4 for Rayleigh, 0 for "gray".  Valid range: 0 - 6

#stuffing all variables into state vector array
x=np.array([Tirr, logKir,logg1, logMet, logCtoO, logPQCarbon,logPQNitrogen, Rp*xRp, Rstar, M, logKzz, fsed,logPbase,logCldVMR, logKcld, logRayAmp, RaySlope])
#gas scaling factors to mess with turning on various species
#set to "0" to turn off a gas. Otherwise keep set at 1
# 0   1    2    3   4    5    6     7    8    9   10    11   12   13    14   15   16   17   18  19
#H2O  CH4  CO  CO2 NH3  N2   HCN   H2S  PH3  C2H2 C2H6  Na    K   TiO   VO   FeH  H    H2   He  mmw
gas_scale=np.array([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]) #can be made free params if desired (won't affect mmw)


# Define Synthetic Data Wavelength Grid 
Define the simulated HST WFC3 wavelength grid.  You could do whatever you want here really. You could add in a STIS simulated wavelength grid as well.  


In [ ]:
#fake data wavelength grid--in microns
wlgrid=np.arange(1.1, 1.6, 0.035)  #lower bound, upper bound, and dlambda--what's here is a "typical" WFC3 binning


# Generate Model Spectrum  

Here we call the forward model routine "fx" (think F(x)) from fm.py.  fx controls the input values and calls the relevent functions to compute the transmission spectrum.  The inputs into fx are the parameter state vector, "x", the simulated data wavelength grid, "wlgrid", the gas scaling factors (for turning off particular gases), "gas_scale", and the correlated-K tables, "xsects".  Fx then returns the simulated model spectrum ($(R_p/R_{\star})^2$) at the native CK-table resolution, "y_mod", the native wavenumber grid, "wno", the data wavelength grid binned model spectrum, "y_binned".  The "atm" array contains the generated temperature-pressure profile and gas mixing ratio profiles generated under the chemically consistent assumption. 

In [ ]:
#calling forward model, fx. This will produce the (Rp/Rstar)^2 spectrum....
y_binned,y_mod,wno,atm=fx(x,wlgrid,gas_scale,xsecs)  #returns model spectrum, wavenumber grid, and vertical abundance profiles from chemistry


# Generate Simulated Data  

Generates the "simulated" data. Effectively, just slapping error bars on top of the binned model spectrum.  A full dataset is defined by its wavelength grid (wlgrid), measured values (y_meas), and the error bars (err).  In this example, 50 ppm error bars were arbitrarily chosen.  Note, the data points aren't "randomized" per noise instantiation (e.g., np.random.normal).  A nice paper by Feng et al. 2018 showed that if you perform a retrieval on many tens of noise-instantiations given the same spectral properties (e.g., error bar) and then average those posteriors, they compare well to the "un-noised" datapoints.  Unsurprising given the Central Limit Theorem...Though, feel free to test this assumption yourself. Great exercise for the "reader".  

In [ ]:
#if generating fake data, do it here
y_meas=y_binned
err=np.zeros(len(y_meas))+50E-6  #50 ppm at all wavelength's error 00 note, no "jiggling"--See Feng+2018 (blah, C.L.T)

# Plotting the Model Spectrum and Simulated Data  

Self-explanatory...

In [ ]:
#finally doing some plotting
#and the usual matplotlib shenanigans
from matplotlib.pyplot import *
from matplotlib.ticker import FormatStrFormatter
ymin=np.min(y_binned)*1E2*0.99
ymax=np.max(y_binned)*1E2*1.01
fig1, ax=subplots()
xlabel('$\lambda$ ($\mu$m)',fontsize=18)
ylabel('(R$_{p}$/R$_{*}$)$^{2} \%$',fontsize=18)
minorticks_on()
errorbar(wlgrid, y_meas*100, yerr=err*100, xerr=None, fmt='Dk')
plot(wlgrid, y_binned*1E2,'ob')
plot(1E4/wno, y_mod*1E2)
ax.set_xscale('log')
ax.set_xticks([0.3, 0.5,0.8,1,1.4, 2, 3, 4, 5])
ax.axis([0.3,5,ymin,ymax])

ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.tick_params(length=10,width=1,labelsize='large',which='major')
savefig('spectrum.pdf',fmt='pdf')
show()
close()


# Plot Model Atmosphere  

Spaghetti plot of the model atmosphere.

In [ ]:
#unpacking variables
#P is in bars
#T is in K
#H2O, CH4,CO,CO2,NH3,Na,K,TiO,VO,C2H2,HCN,H2S,FeH,H2,He are gas mixing ratio profiles
#qc is the condensate abundance profile given an "f_sed" value and cloud base pressure
#r_eff is the effective cloud droplet radius given (see A&M 2001 or Charnay et al. 2017)
#f_r is the mixing ratio array for each of the cloud droplet sizes.
P,T, H2O, CH4,CO,CO2,NH3,Na,K,TiO,VO,C2H2,HCN,H2S,FeH,H2,He,qc,r_eff,f_r=atm


fig2, ax1=subplots()
#feel free to plot whatever you want here....
ax1.semilogx(H2O,P,'b',ls='--',lw=2,label='H2O')
ax1.semilogx(CH4,P,'black',ls='--',lw=2,label='CH4')
ax1.semilogx(CO,P,'g',ls='--',lw=2,label='CO')
ax1.semilogx(CO2,P,'orange',ls='--',lw=2,label='CO2')
ax1.semilogx(NH3,P,'darkblue',ls='--',lw=2,label='NH3')
ax1.semilogx(Na,P,'b',lw=2,label='Na')
ax1.semilogx(K,P,'g',lw=2,label='K')
ax1.semilogx(TiO,P,'k',lw=2,label='TiO')
ax1.semilogx(VO,P,'orange',lw=2,label='VO')
ax1.set_xlabel('Mixing Ratio',fontsize=20)
ax1.set_ylabel('Pressure [bar]',fontsize=20)
ax1.semilogy()
ax1.legend(loc=4,frameon=False)
ax1.axis([1E-9,1,100,1E-7])

#plotting TP profile on other x-axis
ax2=ax1.twiny()
ax2.semilogy(T,P,'r-',lw='4',label='TP')
ax2.set_xlabel('Temperature [K]',color='r',fontsize=20)
ax2.axis([0.8*T.min(),1.2*T.max(),100,1E-6])
for tl in ax2.get_xticklabels(): tl.set_color('r')
savefig('atmosphere.pdf',fmt='pdf')
show()
close()





# Time to Get to the "Retrieval" 

The "retrieval" is performed using the DYNESTY (https://dynesty.readthedocs.io/en/latest/index.html) nested sampling suite.  It's basically like all of the others (e.g., multines, pymultnest, nestle etc.), though it's more flexible in terms of sampling methods optimized for certain numbers of parameters.  This example just uses "generic" settings.

In [ ]:
#set up a "dynest" nested sampling run--see https://dynesty.readthedocs.io/en/latest/index.html
#a super cool useful comparision of all MCMC/Multinests out there...
#http://mattpitkin.github.io/samplers-demo/pages/samplers-samplers-everywhere/
#(feel free to mix and match samplers--probably not a bad idea...)
#for safty, just reloading everything again
import numpy as np
from matplotlib import pyplot as plt
import dynesty
from multiprocessing import Pool
from fm import *
import pickle
#load crosssections between wnomin and wnomax
xsecs=xsects_HST(6000,9400)  #make sure this range is *larger* than the data wavelength grid (but not by too much)

# Defining log-likelihood function
This computes "chi-square" log-likelihood function.  The input value is "theta" which is the same as "x"--the parameter state vector, though just for the parameters we care about. These can be a sub-set of the full "x" defined above.  The first block defines a bunch of parameters--same as what gets passed into fx--to a generic default value.  Thes parameter values get overridden with the values in the "theta (parameters to retrieve) vector. In this particular example only Tirr, logMet, logCtoO, logKzz, fsed ,logPbase,logCldVMR,xRp are retrieved.  All other values passed into fx are assigned the other values.  

In [ ]:
#defining log-likelihood function
# log-likelihood
def loglike(theta):

    #setting default parameters---will be fixed to these values unless replaced with 'theta'
    #planet/star system params--xRp is the "Rp" free parameter, M right now is fixed, but could be free param
    Rp= 1.10   # Planet radius in Jupiter Radii--this will be forced to be 1 bar radius--arbitrary (scaling to this is free par)
    Rstar=1.20  #Stellar Radius in Solar Radii
    M =1.0   #Mass in Jupiter Masses
    xRp=1.0  #scaling factor to Radius
    #TP profile params (3--Guillot 2010, Parmentier & Guillot 2013--see Line et al. 2013a for implementation)
    Tirr=1400     #Irradiation temperature as defined in Guillot 2010
    logKir=-0.5  #TP profile IR opacity controlls the "vertical" location of the gradient
    logg1=-1     #single channel Vis/IR opacity. Controls the delta T between deep T and TOA T

    #Composition parameters---assumes "chemically consistent model" described in Kreidberg et al. 2015
    logMet=0.0  #.   #Metallicity relative to solar log--solar is 0, 10x=1, 0.1x = -1: valid range is -1.5 - 3.0
    logCtoO=-0.26   #log C-to-O ratio: log solar is -0.26: valid range is -1.0 - 0.3 
    logPQCarbon=-5.5  #CH4, CO, H2O Qunech pressure--forces CH4, CO, and H2O to constant value at quench pressure value: valid range -6.0 - 1.5
    logPQNitrogen=-5.5  #N2, NH3 Quench pressure--forces N2 and NH3 to ""  

    #Ackerman & Marley 2001 Cloud parameters--physically motivated with Mie particles
    logKzz=9 #log Kzz (cm2/s)--valid range: 2 - 11 -- higher values make larger particles
    fsed=1.0 #sediminetation efficiency--valid range: 0.5 - 5--lower values make "puffier" more extended cloud 
    logPbase=1.5  #cloud base pressure--valid range: -6.0 - 1.5
    logCldVMR=-15.0 #cloud condensate base mixing ratio (e.g, see Fortney 2005)--valid range: -15 - -2.0

    #simple 'grey+power-law' parameters 
    logKcld = -40  #uniform in altitude and in wavelength "grey" opacity (it's a cross-section)--valid range: -50 - -10 
    logRayAmp = -30  #power-law haze amplitude (log) as defined in des Etangs 2008 "0" would be like H2/He scat--valid range: -30 - 3 
    RaySlope = 0  #power law index 4 for Rayleigh, 0 for "gray".  Valid range: 0 - 6


    #unpacking parameters to retrieve
    Tirr, logMet, logCtoO, logKzz, fsed ,logPbase,logCldVMR,xRp=theta
   
    ##all values required by forward model go here--even if they are fixed
    x=np.array([Tirr, logKir,logg1, logMet, logCtoO, logPQCarbon,logPQNitrogen, Rp*xRp, Rstar, M, logKzz, fsed,logPbase,logCldVMR, logKcld, logRayAmp, RaySlope])
    gas_scale=np.array([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1., 1.])
    y_binned,y_mod,wno,atm=fx(x,wlgrid,gas_scale,xsecs)


    loglikelihood=-0.5*np.sum((y_meas-y_binned)**2/err**2)  #your typical "quadratic" or "chi-square"
    return loglikelihood


# Defining the Prior Function
Multinest uses this "cube" concept whereby all of the parameter values are transformed on the interval [0,1]. This makes the sampling more-or-less scale indepenent.  The prior_transform function maps those back onto the actual parameter value ranges we want.  In this case, the prior ranges are all uniform over the range as defined here. Live points will be drawn from this hypercube uniformly.  These "live point" parameter values are then assesed by the log-likelihood function.    

In [ ]:
#defining prior cube (cube is a standard multi-nest way of doing priors)
def prior_transform(utheta):
    Tirr, logMet, logCtoO, logKzz, fsed ,logPbase,logCldVMR,xRp=utheta
    #uniform prior ranges--each "variable", say Tirr is sampled over the interval [0,1]--the numbers here transform that
    Tirr = 2600 * Tirr + 400  #Tirr uniform from 400 - 3000K (add the lower value to the multiplier to get upper bound)
    logMet= 4.5*logMet-1.5  #[M/H] from -1.5 - 3.0 (0.03x - 1000x)
    logCtoO=1.3*logCtoO-1
    logKzz=6*logKzz+5
    fsed=3.5*fsed+0.5
    logPbase=7.5*logPbase-6.0
    logCldVMR=8*logCldVMR-10
    xRp=1*xRp+0.5

    return Tirr, logMet, logCtoO, logKzz, fsed ,logPbase,logCldVMR,xRp




# Loading in Data and Setting up Nested Sampling Parameters
This segment first loads in the "3 column" data file...Here it is just a pickle, but this could be replaced with a 3-column ascii file read in--wavelength grid, data values, error bars.  The other knobs are self-explanatory.  The number of live points is "problem dependent".  Some problems can get away with less. I wouldn't go below 100.  It's safer to use more (e.g., just like walkers in emcee).  For "real science" I prefer 1000+ live points to make sure the posterior is well sampled and no modes are missed.  Of course, this takes longer to run, but no sense in getting the the wrong answer faster!

In [ ]:
#setting up other dynesty run params and loading in the data
wlgrid, y_meas, err=pickle.load(open('data.pic','rb'))
outname='dyn_output_500LP.pic'  #dynesty output file name (saved as a pickle)
Nparam=8  #number of parameters--make sure it is the same as what is in prior and loglike
Nproc=4  #number of processors for multi processing--best if you can run on a 12 core+ node or something
Nlive=500 #number of nested sampling live points


# Running Nested Sampler
This calls the Nested Sampler function to compute the posterior.  This may take a few hours depending on the number of parameters and number of live points.

In [ ]:
#running the "standard" nested sampler.  Again, see https://dynesty.readthedocs.io/en/latest/index.html for details
#depending on the number of live points (I like lots, usually 1000+, but I'm paranoid), number of params, and number of
#processors on your computer, this could take some time. -- your computer will make loud noises..1-2 hours with 4 cores
import time
pool = Pool(processes=Nproc)
dsampler = dynesty.NestedSampler(loglike, prior_transform, ndim=Nparam,
                                        bound='multi', sample='auto', nlive=Nlive,
                                        update_interval=3., pool=pool, queue_size=Nproc)
#this executes and runs it
t1=time.time()
dsampler.run_nested()
t2=time.time()
print("Run Time:", t2-t1)
#extracting results from sampler object
dres = dsampler.results
#dumping as a pickle
pickle.dump(dres,open(outname,'wb'))  
#some real time output will pop up down here

# Plotting Corner Plot
Plots the corner plots and spectral fit plot. Note: All of the plotting below can be used for a "previously" generated run. Just start here and load in the sampler output.

In [ ]:
#plotting dynest runs corner plot
from matplotlib import pyplot as plt
from dynesty import plotting as dyplot
import pickle


truth=[1300,0,-0.26,9, 1, 1.5, -15, 1]  #values to overplot on histograms
labels=['Tirr', 'logMet', 'logCtoO', 'logKzz', 'fsed' ,'logPbase','logCldVMR','xRp']

#import past run 
#samples=pickle.load(open('dyn_output_100LP.pic','rb')) #an example 100 live point run
samples=pickle.load(open('dyn_output_500LP.pic','rb')) 
#samples=pickle.load(open('dyn_output_1000LP.pic','rb'))  #an example 1000 live point run

#printing evidence:
print('ln(Z)= ', samples.logz[-1])

# corner plot
fig, axes = dyplot.cornerplot(samples,smooth=0.05, color='red',show_titles=True, labels=labels, truths=truth,title_kwargs={'y': 1.04}, fig=plt.subplots(8, 8, figsize=(12, 12)))
plt.savefig('stair_pairs.pdf',fmt='pdf')
plt.show()
plt.close()




# Plotting Spectral Fits
Generating spectra from parameters of a subset of samples drawn from the posterior.  These spectra, as always, are then summarizes with their median, 1-, 2-, sigma confidence intervals.  

In [ ]:
import numpy as np
xsecs=xsects_HST(2000, 30000)

Nspectra=200

#loading in data again just to be safe
wlgrid, y_meas, err=pickle.load(open('data.pic','rb'))


#setting up default parameter values--SET THESE TO SAME VALUES AS IN LOG-LIKE FUNCTION
#planet/star system params--xRp is the "Rp" free parameter, M right now is fixed, but could be free param
Rp= 1.10   # Planet radius in Jupiter Radii--this will be forced to be 1 bar radius--arbitrary (scaling to this is free par)
Rstar=1.20  #Stellar Radius in Solar Radii
M =1.0   #Mass in Jupiter Masses
xRp=1.0  #scaling factor to Radius

#TP profile params (3--Guillot 2010, Parmentier & Guillot 2013--see Line et al. 2013a for implementation)
Tirr=1400     #Irradiation temperature as defined in Guillot 2010
logKir=-0.5  #TP profile IR opacity controlls the "vertical" location of the gradient
logg1=-1     #single channel Vis/IR opacity. Controls the delta T between deep T and TOA T

#Composition parameters---assumes "chemically consistent model" described in Kreidberg et al. 2015
logMet=0.0  #.   #Metallicity relative to solar log--solar is 0, 10x=1, 0.1x = -1: valid range is -1.5 - 3.0
logCtoO=-0.26   #log C-to-O ratio: log solar is -0.26: valid range is -1.0 - 0.3 
logPQCarbon=-5.5  #CH4, CO, H2O Qunech pressure--forces CH4, CO, and H2O to constant value at quench pressure value: valid range -6.0 - 1.5
logPQNitrogen=-5.5  #N2, NH3 Quench pressure--forces N2 and NH3 to ""  

#Ackerman & Marley 2001 Cloud parameters--physically motivated with Mie particles
logKzz=9 #log Kzz (cm2/s)--valid range: 2 - 11 -- higher values make larger particles
fsed=1.0 #sediminetation efficiency--valid range: 0.5 - 5--lower values make "puffier" more extended cloud 
logPbase=1.5  #cloud base pressure--valid range: -6.0 - 1.5
logCldVMR=-15.0 #cloud condensate base mixing ratio (e.g, see Fortney 2005)--valid range: -15 - -2.0

#simple 'grey+rayleigh' parameters just in case you don't want to use a physically motivated cloud
#(most are just made up anyway since we don't really understand all of the micro-physics.....)
logKcld = -40  #uniform in altitude and in wavelength "grey" opacity (it's a cross-section)--valid range: -50 - -10 
logRayAmp = -30  #power-law haze amplitude (log) as defined in des Etangs 2008 "0" would be like H2/He scat--valid range: -30 - 3 
RaySlope = 0  #power law index 4 for Rayleigh, 0 for "gray".  Valid range: 0 - 6



#weighting the posterior samples for appropriate random drawing
from dynesty import utils as dyfunc
samp, wts = samples.samples, np.exp(samples.logwt - samples.logz[-1])
samples2 = dyfunc.resample_equal(samp, wts)

#choosing random indicies to draw from properly weighted posterior samples
draws=np.random.randint(0, samples2.shape[0], Nspectra)
Nwno_bins=xsecs[2].shape[0]
y_mod_array=np.zeros((Nwno_bins, Nspectra))
y_binned_array=np.zeros((len(wlgrid), Nspectra))

for i in range(Nspectra):
    print(i)
    #make sure this is the same as in log-Like
    Tirr, logMet, logCtoO, logKzz, fsed ,logPbase,logCldVMR,xRp=samples2[draws[i],:]
    x=np.array([Tirr, logKir,logg1, logMet, logCtoO, logPQCarbon,logPQNitrogen, Rp*xRp, Rstar, M, logKzz, fsed,logPbase,logCldVMR, logKcld, logRayAmp, RaySlope])
    print(samples.samples[draws[i],:])
    gas_scale=np.array([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1., 1.])
    y_binned,y_mod,wno,atm=fx(x,wlgrid,gas_scale,xsecs)
    y_mod_array[:,i]=y_mod
    y_binned_array[:,i]=y_binned
    
#saving these arrays since it takes a few minutes to generate    
pickle.dump([wlgrid, y_meas, err, y_binned_array, wno, y_mod_array],open('spectral_samples.pic','wb'))



In [ ]:
wlgrid, y_meas, err, y_binned_array, wno, y_mod_array=pickle.load(open('spectral_samples.pic','rb'))
y_median=np.zeros(wno.shape[0])
y_high_1sig=np.zeros(wno.shape[0])
y_high_2sig=np.zeros(wno.shape[0])
y_low_1sig=np.zeros(wno.shape[0])
y_low_2sig=np.zeros(wno.shape[0])

for i in range(wno.shape[0]):
    percentiles=np.percentile(y_mod_array[i,:],[4.55, 15.9, 50, 84.1, 95.45])
    y_low_2sig[i]=percentiles[0]
    y_low_1sig[i]=percentiles[1]
    y_median[i]=percentiles[2]
    y_high_1sig[i]=percentiles[3]
    y_high_2sig[i]=percentiles[4]
    
    
from matplotlib.pyplot import *
from matplotlib.ticker import FormatStrFormatter

ymin=np.min(y_meas)*1E2*0.98
ymax=np.max(y_meas)*1E2*1.02
fig1, ax=subplots()
xlabel('$\lambda$ ($\mu$m)',fontsize=14)
ylabel('(R$_{p}$/R$_{*}$)$^{2} \%$',fontsize=14)
minorticks_on()


#for i in range(20): plot(wlgrid, y_binned_array[:,i]*100.,alpha=0.5,color='red')
#for i in range(20): plot(1E4/wno, y_mod_array[:,i]*100.,alpha=0.5,color='red')

fill_between(1E4/wno[::-1],y_low_2sig[::-1]*100,y_high_2sig[::-1]*100,facecolor='r',alpha=0.5,edgecolor='None')  
fill_between(1E4/wno[::-1],y_low_1sig[::-1]*100,y_high_1sig[::-1]*100,facecolor='r',alpha=1.,edgecolor='None')  


errorbar(wlgrid, y_meas*100, yerr=err*100, xerr=None, fmt='Dk')
plot(1E4/wno, y_median*1E2)
ax.set_xscale('log')
ax.set_xticks([0.3, 0.5,0.8,1,1.4, 2, 3, 4, 5])
ax.axis([0.3,5.0,ymin,ymax])

ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.tick_params(length=5,width=1,labelsize='small',which='major')
savefig('spectrum_fits.pdf',fmt='pdf')
show()
close()

    